# ANSWER KEY: Lesson 1 Exercise 2: Creating a Table with Apache Cassandra


cassandra logo

### In this exercise we are going to walk through the basics of creating a table in Apache Cassandra, inserting rows of data, and doing a simple SQL query to validate the information. 

### Fill in the code where you see #####

#### We will use a python wrapper/ python driver called cassandra to run the Apache Cassandra queries. This library should be preinstalled, but to install this library in the future you can run this command in a notebook to install locally: 
! pip install cassandra-driver
#### More documentation can be found here:  https://datastax.github.io/python-driver/

#### Import Apache Cassandra python package

In [17]:
import cassandra
!pip freeze 


alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
anaconda-client==1.7.2
anaconda-navigator==2.0.3
anaconda-project @ file:///tmp/build/80754af9/anaconda-project_1610472525955/work
anyio @ file:///opt/concourse/worker/volumes/live/64740ac7-3a9c-4fbb-6685-a51c4ff8b4ca/volume/anyio_1617783319350/work/dist
appdirs==1.4.4
applaunchservices==0.2.1
appnope @ file:///opt/concourse/worker/volumes/live/5f13e5b3-5355-4541-5fc3-f08850c73cf9/volume/appnope_1606859448618/work
appscript @ file:///opt/concourse/worker/volumes/live/82e8b4c7-2416-4d10-509e-144ca79d9b1d/volume/appscript_1611426996703/work
argh==0.26.2
argon2-cffi @ file:///opt/concourse/worker/volumes/live/d733ceb5-7f19-407b-7da7-a386540ab855/volume/argon2-cffi_1613037492998/work
asn1crypto @ file:///tmp/build/80754af9/asn1crypto_1596577642040/work
astroid @ file:///opt/concourse/worker/volumes/live/343a8902-287c-47fb-6db8-923a63364302/volume/astroid_1613500849157/work
astropy @ file:///opt/concourse/worker/volumes/liv

### First let's create a connection to the database
This connects to our local instance of Apache Cassandra. This connection will reach out to the database and insure we have the correct privileges to connect to this database. Once we get back the cluster object, we need to connect and that will create our session that we will use to execute queries.
#### Note 1: This block of code will be standard in all notebooks

In [18]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)
 

('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(61, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})


### Let's Test our Connection 
We are trying to do a `select *` on a table we have not created yet. We should expect to see a nicely handled error. 

In [19]:
try: 
    session.execute("""select * from music_libary""")
except Exception as e:
    print(e)
 

name 'session' is not defined


### Let's create a keyspace to do our work in 
Note: Ignore the Replication Stratgety and factor information for now. Those will be discussed later. Just know that on a one node local instance this will be the strategy and replication factor. 

In [20]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

name 'session' is not defined


#### Connect to our Keyspace. Compare this to how we had to create a new session in PostgreSQL.  

In [21]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

name 'session' is not defined


## Let's imagine we would like to start creating a Song Library of all the songs we own. Each song has a lot of information we could add to the song library table, but we will just start with name of the song, artist name, year, album it was from, and if it was a single.

`song title
artist
year
album
single`


## But ...STOP


### We are working with Apache Cassandra a NoSQL database. We can't model our data and create our table with out more information.

# What queries will I be performing on this data?

### In this case I would like to be able to get every song by a certain artist that was released in a particular year. 


`select * from songs WHERE YEAR=1970 AND artist_name='The Beatles'`

### Because of this I need to be able to do a WHERE on YEAR. YEAR will become my partition key, and artist name will be my clustering column to make each Primary Key unique. Remember there are no duplicates in Apache Cassandra. 

### Now to translate this information into a Create Table Statement. 

#### More infomration on Data Types can be found here: https://datastax.github.io/python-driver/

In [22]:
query = "CREATE TABLE IF NOT EXISTS songs "
query = query + "(year int, song_title text, artist_name text, album_name text, single boolean, PRIMARY KEY (year, artist_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)


name 'session' is not defined


### No error was found, but let's check to ensure our table was created.  `select count(*)` which should return 0 as we have not inserted any rows. 

Note: Depending on the version of Apache Cassandra you have installed, this might throw an "ALLOW FILTERING" error instead of a result of "0". This is to be expected, as this type of query should not be performed on large datasets, we are only doing this for the sake of the demo.

In [23]:
query = "select count(*) from songs"
try:
    count = session.execute(query)
except Exception as e:
    print(e)
    
print(count.one())

name 'session' is not defined


NameError: name 'count' is not defined

### Let's insert two rows
`First Row:  "Across The Universe", "The Beatles", "1970", "False", "Let It Be"`

`Second Row: "The Beatles", "Think For Yourself", "False", "1965", "Rubber Soul"`

In [8]:
query = "INSERT INTO songs (year, song_title, artist_name, album_name, single)"
query = query + " VALUES (%s, %s, %s, %s, %s)"

try:
    session.execute(query, (1970, "Across The Universe", "The Beatles", "Let It Be", False))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "Think For Yourself", "The Beatles", "Rubber Soul", False))
except Exception as e:
    print(e)

name 'session' is not defined
name 'session' is not defined


### Validate your data was inserted into the table.
Note: The for loop is used for printing the results. If executing queries in the cqlsh, this would not be required.

Note: Depending on the version of Apache Cassandra you have installed, this might throw an "ALLOW FILTERING" error instead of print the 2 rows we just inserted. This is to be expected, as this type of query should not be performed on large datasets, we are only doing this for the sake of the demo.

In [9]:
query = 'SELECT * FROM songs'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.year, row.album_name, row.artist_name)

name 'session' is not defined


NameError: name 'rows' is not defined

### Let's Validate our Data Model with our orignal query.

`select * from songs WHERE YEAR=1970 AND artist_name='The Beatles'`

In [10]:
query = "select * from songs WHERE YEAR=1970 AND artist_name='The Beatles'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.year, row.album_name, row.artist_name)

name 'session' is not defined


NameError: name 'rows' is not defined

### And Finally close the session and cluster connection

In [11]:
session.shutdown()
cluster.shutdown()

NameError: name 'session' is not defined